In [1]:
import glob
import pandas as pd
import pathlib

In [2]:
all_cools = glob.glob(f'/data/female-amb/impute/merged-25k/*.Q.cool')
ct_age = [path.split('/')[-1].split('.Q')[0] for path in all_cools]

In [3]:
df = pd.DataFrame({'ct_age':ct_age,'all_cools':all_cools})

In [5]:
df['ct'] = df['ct_age'].apply(lambda x: x.split('.')[0])

In [14]:
all_cts = []
for ct, tmpdf in df.groupby('ct'):
    tmpdf = tmpdf[['ct_age','all_cools']]
    tmpdf.to_csv(f'bulk_table/{ct}.tsv', sep = '\t', header = None, index= None)
    all_cts.append(ct)

In [15]:
output_dir = 'hicluster_bulk_domain'
p = pathlib.Path(output_dir)
p.mkdir(parents=True, exist_ok=True)

In [19]:
cpu = 15

In [21]:
with open("boundary.sh", 'w') as file:
    command_count = 0
    for ct in all_cts:
        cell_table_path = f'bulk_table/{ct}.tsv'
        output_prefix = f'hicluster_bulk_domain/{ct}'
        command = f"hicluster domain \
--cell_table_path {cell_table_path} \
--output_prefix {output_prefix} \
--resolution 25000 \
--save_count \
--window_size 10 \
--cpu {cpu} &"
        file.write(f"{command}\n")
        
        command_count += 1
        if command_count == 3:
            file.write("wait\n")
            command_count = 0
